<a href="https://colab.research.google.com/github/nspiegeln/iml_2022/blob/main/task4_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import numpy as np
from torch import nn
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
#load data
from google.colab import files
uploaded = files.upload()

In [ ]:
df_train= pd.read_csv(path + "train_features.csv")
df_train_labels= pd.read_csv(path + "train_labels.csv")
df_pretrain= pd.read_csv(path + "pretrain_features.csv")
df_pretrain_labels= pd.read_csv(path + "pretrain_labels.csv")
df_test= pd.read_csv(path + "test_features.csv")

In [ ]:

df_train_values = df_train.iloc[:,2::]
df_pretrain_values = df_pretrain.iloc[:,2::]

df_train_lab = df_train_labels.iloc[:,1]
df_pretrain_lab = df_pretrain_labels.iloc[:,1]

train = df_train_values.values #dataset with known homo-lumo gap
pretrain = df_pretrain_values.values #dataset with known lumo value

train_labels = df_train_lab.values
pretrain_labels = df_pretrain_lab.values


In [ ]:
x_pretrain, x_pretest, y_pretrain, y_pretest = train_test_split(pretrain, pretrain_labels, test_size = 0.9)

In [ ]:
# PRE-PROCESSING of features according to autoencoder
# pre-train set

#hyperparameters
dim = 200
epochs = 5 # best results for epohcs= 5, anything larger and the loss starts increasing again

# create encoder and decoder to reduce dimensionality of features
class Autoencoder(Model):
  def __init__(self, dim):
    super(Autoencoder, self).__init__()
    self.dim = dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(1000, activation='sigmoid')
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(dim)

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder.fit(x_pretrain, x_pretrain, epochs=epochs, shuffle=True, validation_data=(x_pretest, x_pretest))

In [ ]:
x_encoded_pretrain = autoencoder.encoder(x_pretrain).numpy()
x_encoded_pretest = autoencoder.encoder(x_pretest).numpy()

In [ ]:
# hyperparameters
output_dim = 1
epochs = 100
# predict one LUMO label
class Predictor(Model):
    def __init__(self, dim):
      super(Predictor, self).__init__()
      self.output_dim = output_dim   
      self.predictor = tf.keras.Sequential([
        layers.Flatten(),
        layers.Dense(50, activation='relu'),
        layers.Dense(output_dim, activation='relu'),
      ])

    def call(self, x):
      predictor = self.predictor(x)
      return predictor
    
predictor = Predictor(output_dim)
predictor.compile(optimizer='adam', loss=losses.MeanSquaredError())
predictor.fit(x_encoded_pretrain, y_pretrain, epochs=epochs, shuffle=True, validation_data=(x_encoded_pretest, y_pretest))